# Using DeepLabCut Toolbox under Python Package

安裝: `!pip install deeplabcut`

主要參考此篇說明: [Using DeepLabCut for 3D markerless pose estimation across species and behaviors](https://www.biorxiv.org/content/10.1101/476531v1)

In [1]:
"""
由於 hub 環境不支援 GUI，參考說明文件 P.7 設定環境變數。

  * 需要先在支援 GUI 環境完成 labelled data

"""
%env DLClight True
import os
from pathlib import Path
import deeplabcut

env: DLClight=True


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


DLC loaded in light mode; you cannot use the labeling GUI!
DLC loaded in light mode; you cannot use the relabeling GUI!


## 建立新專案或是使用現有專案設定

新專案會建好專案使用的目錄:
  - dlc-models: 存放 train/test feature detection 的設定檔及 train 過的 model
  - labeled-data: 透過 **label_frames** 的資料存放處
  - training-datasets
  - videos: 專案使用的 videos
  
需要自行修改專案目錄下的 config.yaml:
  - bodyparts: 要 track 的目標點
  - numframes2pick: 要用多少 frames 來 training
  - start, stop, x1, x2, y1, y2: 裁切

In [2]:
task='BeetleTracking' # experiment Task
experimenter='Lindo' # experimenter
# Enter the paths of your videos you want to grab frames from.
video=['../datasets/beetle-tracking/Detection_data/2016-10-14_x264.avi'] 

''' 建立新專案 '''
#path_config_file = deeplabcut.create_new_project(task, experimenter, video, working_directory='',copy_videos=True)
''' 使用舊的專案 '''
path_config_file = os.path.join(os.getcwd(),'BeetleTracking-Lindo-2019-03-26/config.yaml')

## 從影片截取 frames
好的 datasets 要涵蓋所有 postures 的變化以及亮度變化等，根據原論文的測試，建議數量在 100-200 frames 較佳

In [3]:
%matplotlib inline

''' 演算法可以用 'uniform' 或是 'kmeans' '''
deeplabcut.extract_frames(path_config_file,'automatic','uniform', crop=False)

/home/jovyan/.local/lib/python3.6/site-packages/deeplabcut/generate_training_dataset/frame_extraction.py:100: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


Config file read successfully.
Do you want to extract (perhaps additional) frames for video: /home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/videos/2016-10-14_x264.avi ?


yes/no yes


ZeroDivisionError: float division by zero

## Label frames
這步目前是透過支援 GUI 的環境完成

In [5]:
if os.environ.get('DLClight', default=False) == 'False':
    %gui wx
    deeplabcut.label_frames(path_config_file)

**檢查 labels**

In [4]:
''' 基本上是檢查檔案格式正確與否，以及標記的點是否在影像範圍內。 '''
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Lindo.
/home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/labeled-data/2016-10-14_x264_labeled  already exists!
Attention: /home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/labeled-data/2016-10-14_x264 does not appear to have labeled data!
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


## 建立 training dataset
在 training 前執行。大致是將所有 labeled image 用 hd5 存放 (後續用 pandas dataframes 存取)

In [5]:
deeplabcut.create_training_dataset(path_config_file)

/home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/training-datasets/iteration-1/UnaugmentedDataSet_BeetleTrackingMar26  already exists!
It appears that the images were labeled on a Windows system, but you are currently trying to create a training set on a Unix system. 
 In this case the paths should be converted. Do you want to proceed with the conversion?


yes/no yes


Annotation data converted to unix format...
/home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1  already exists!
/home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1//train  already exists!
/home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


/home/jovyan/.local/lib/python3.6/site-packages/deeplabcut/generate_training_dataset/trainingsetmanipulation.py:328: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  docs.append(yaml.load(raw_doc))


## 開始 training
hub環境，基本上跑完 103000 個 iterations 需時約 28 hr

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=1000, saveiters=1000)

## 開始 evaluating
評估 training 結果，在 config 檔內可設定 p-cutoff 值；結果會存放在 **evaluation-results** 目錄

In [6]:
''' funcation 內會選取最大的 iterations 來做評估 '''
deeplabcut.evaluate_network(path_config_file)

/home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/evaluation-results/  already exists!
/home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/evaluation-results/iteration-1/BeetleTrackingMar26-trainset95shuffle1  already exists!
Running  DeepCut_resnet50_BeetleTrackingMar26shuffle1_1030000  with # of trainingiterations: 1030000
This net has already been evaluated!
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


/home/jovyan/.local/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/config.py:43: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


## 分析目標 video
等於做 prediction，結果會存成 hd5 檔。

In [7]:
''' 為了方便自己閱讀，加上 save_as_csv=True 存一份 csv '''
videofile_path = os.path.join(os.getcwd(),'BeetleTracking-Lindo-2019-03-26/videos/[CH01]2016-10-1419.20.00_x264_m_04.avi')
deeplabcut.analyze_videos(path_config_file, [videofile_path],save_as_csv=True)

/home/jovyan/.local/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/config.py:43: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


Using snapshot-1030000 for model /home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1
INFO:tensorflow:Restoring parameters from /home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1/train/snapshot-1030000


INFO:tensorflow:Restoring parameters from /home/jovyan/jt072-09-biodiv-beetle/DeepLabCut/examples/BeetleTracking-Lindo-2019-03-26/dlc-models/iteration-1/BeetleTrackingMar26-trainset95shuffle1/train/snapshot-1030000
  0%|          | 0/1000 [00:00<?, ?it/s]

Starting to analyze %  /home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/videos/[CH01]2016-10-1419.20.00_x264_m_04.avi
Loading  /home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/videos/[CH01]2016-10-1419.20.00_x264_m_04.avi
Duration of video [s]:  66.67 , recorded with  15.0 fps!
Overall # of frames:  1000  found with (before cropping) frame dimensions:  1280 720
Starting to extract posture


1010it [00:51, 19.21it/s]                          

Detected frames:  1000
Saving results in /home/jovyan/Project_JT072_09_Biodiv-Beetle/DeepLabCut/ByPackage/BeetleTracking-Lindo-2019-03-26/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## 建立 labeled video

In [14]:
deeplabcut.create_labeled_video(path_config_file,[videofile_path])